# SeilabThermal notebook example

In this example, a rule based controller is used to control the model "SeilabThermal-v0". At first we import the controller:

In [1]:
from energym.examples.Controller import LabController

Next, we import Energym and create the simulation environment by specifying the model, a weather file and the number of simulation days.

In [2]:
import energym

weather = "ESP_CT_Barcelona"
env = energym.make("SeilabThermal-v0", weather=weather, simulation_days=30)

C:\Users\bsl\AppData\Local\Continuum\anaconda3\envs\data_program\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


[OK] fmi2Instantiate: The Resource location of FMU with instance name %s is %s.

[WARNING] fmi2Instantiate: Argument loggingOn is set to %d
. This is not supported. loggingOn will default to '0'.

[OK] The current working directory is %s

[OK] fmi2Instantiate: Path to fmuUnzipLocation is not %s.

[OK] fmi2Instantiate: Path to fmuUnzipLocation is not %s.

[OK] fmi2Instantiate: Path to fmuUnzipLocation is not %s.

[OK] fmi2Instantiate: Path to fmuUnzipLocation %s

[OK] fmi2Instantiate: Path to fmuResourceLocation %s

[OK] Command executes to copy content of resources folder: %s

[OK] fmi2Instantiate: Path to model description file is %s.

[OK] fmi2Instantiate: The FMU modelIdentifier is %s.

[OK] fmi2Instantiate: The FMU modelGUID is %s.

[OK] fmi2Instantiate: Slave %s is instantiated.

[OK] fmi2Instantiate: Instantiation of %s succeded.



The control inputs can be inspected using the `get_inputs_names()` method and to construct a controller, we pass the list of inputs and further parameters. This controller determines inputs to get close to the temperature setpoints and uses fixed setpoints during the night.

In [4]:
inputs = env.get_inputs_names()
print(inputs)
controller = LabController(control_list=inputs, lower_tol=0.3, upper_tol=0.8, nighttime_setback=True, nighttime_start=18, nighttime_end=6, nighttime_temp=18)

['Bd_Ch_EVBat_sp', 'Bd_Pw_Bat_sp', 'Bd_T_HP_sp', 'HVAC_onoff_HP_sp', 'P1_T_Tank_sp', 'P1_T_Thermostat_sp', 'P2_T_Tank_sp', 'P2_T_Thermostat_sp', 'P3_T_Tank_sp', 'P3_T_Thermostat_sp', 'P4_T_Tank_sp', 'P4_T_Thermostat_sp']


To run the simulation, a number of steps is specified (here 480 steps per day for 10 days) and the obtained control inputs are passed to the simulation model with the `step()` method. To generate some plots later on, we save all the inputs and outputs in lists.

In [38]:
steps = 480*2
out_list = []
outputs = env.get_output()
controls = []
hour = 0
for i in range(steps):
    control = controller.get_control(outputs, 21, hour)
    control['P1_T_Tank_sp'] = [40.0] 
    control['P2_T_Tank_sp'] = [40.0]
    control['P3_T_Tank_sp'] = [40.0]
    control['P4_T_Tank_sp'] = [40.0]
    control['Bd_Ch_EVBat_sp'] = [1.0]
    control['Bd_DisCh_EVBat_sp'] = [0.0]
    control['HVAC_onoff_HP_sp']= [1.0]
    control['Bd_T_HP_sp']=[45]
    controls +=[ {p:control[p][0] for p in control} ]
    outputs = env.step(control)
    _,hour,_,_ = env.get_date()
    out_list.append(outputs)

Since the inputs and outputs are given as dictionaries and are collected in lists, we can simply load them as a pandas.DataFrame.

In [39]:
import pandas as pd
out_df = pd.DataFrame(out_list)
cmd_df = pd.DataFrame(controls)

To generate plots, we can directly get the data from the DataFrames, by using the key names. Displayed are the zone temperatures and the setpoints determined by the controller for zone 1, the external temperature, and the total power demand.

In [40]:
import matplotlib.pyplot as plt
%matplotlib notebook

f, (ax1,ax2,ax3) = plt.subplots(3,figsize=(10,15))#


ax1.plot(out_df['Z01_T'], 'r')
ax1.plot(out_df['P1_T_Thermostat_sp_out'], 'b--')
ax1.set_ylabel('Temp')
ax1.set_xlabel('Steps')

ax2.plot(out_df['Ext_T'], 'r')
ax2.set_ylabel('Temp')
ax2.set_xlabel('Steps')

ax3.plot(out_df['Fa_Pw_All'], 'g')
ax3.set_ylabel('Power')
ax3.set_xlabel('Steps')

plt.subplots_adjust(hspace=0.4)

plt.show()

<IPython.core.display.Javascript object>

To end the simulation, the `close()` method is called. It deletes files that were produced during the simulation and stores some information about the simulation in the *runs* folder.

In [7]:
env.close()

[OK] fmi2Terminate: fmiFreeInstanceSlave must be called to free the FMU instance.

[OK] fmi2FreeInstance: The function fmi2FreeInstance of instance %s is executed.

[OK] freeInstanceResources: %s will be freed.

